# Workflow

In [ ]:
# Collect Parameters
params = dict()

In [ ]:
from pathlib import Path
from enum import Enum
from ess.nmx.loader import InputFileName

class FileNames(Enum):
    """Enum class for file names."""
    file1 = '2e11.h5'
    file2 = 'x0_3d.h5'
    file3 = 'xe_0_NMX.h5'
    file4 = 'xe_20_NMX.h5'
    file5 = 'flip_det.h5'
    file6 = 'flip_4.h5'
    file7 = 'ye_0_NMX.h5'
    file8 = 'xe1_12_NMX/xe1_12_no_chunk.h5'
    file9 = '1d.h5'
    file10 = 'xflat20.h5'
    file11 = 'small_vis/flat.nxs'
    file12 = 'small_vis/shift_down.nxs'
    file13 = 'mc2_7_2_test.h5'
    file14 = 'mc2_new_comp.nxs'
    file15 = 'xflat20.h5'
    file16 = 'flat_h_r.h5'
    file17 = 'flat_v_u.h5'
    file18 = 'NMX_long.h5'
    file19 = 'pulse5.h5'
    file20 = 'pulse5_y.h5'
    file21 = 'wavelength.h5'
    file22 = 'wavelength_short1.h5'
    file23 = 'pulse5_z.h5'
    file24 = 'nexus-v3_2-with_detectors_stageZ_and_one_axis5.h5'
    file25 = 'xflat20.h5'

fname = 23
data_dir = Path('/home/sunyoung/ESS/data/nmx/')

file_name = getattr(FileNames, f"file{fname}").value
file_path = data_dir / Path(file_name)
f_vis_path = data_dir / Path(file_name.replace('.h5', '_vis.nxs'))

params[InputFileName] = str(file_path.absolute())
print(f"Read from: {file_path}")
print(f"Write results into: {f_vis_path}")

In [ ]:
# f=snx.File(filename)

In [ ]:
from ess.nmx.detector import NumberOfDetectors, MaxNumberOfPixelsPerAxis, PixelStep
from ess.nmx.loader import MaximumPropability
from ess.nmx.reduction import TimeBinStep

# Number of pixels per detectpr dimension
params[MaxNumberOfPixelsPerAxis] = 1280
params[PixelStep] = 100

# Number of time bins 
params[TimeBinStep] = 1

# Number of detectors
params[NumberOfDetectors] = 1

# Normalising highest prop-ability to 
params[MaximumPropability] = 100000

print(f"Data points in output: ", params[MaxNumberOfPixelsPerAxis]**2*3)

Pretreatment to make it faster <br>
not needed with new McStas versions

```bash
h5repack -l CHUNK=1024x6 2e11.h5 2e11-rechunk.h5
# or
h5repack -l CHUNK=NONE 2e11.h5 2e11-nochunk.h5
# or
h5repack -l CHUNK=1024x6 mccode.h5 mccode-nochunk.h5 
```

Test data filename must be the exact file to use for this step of data reduction.

In [ ]:
import scipp as sc

# Parameters to load the file:
sc.DataGroup({param_t.__name__: val for param_t, val in params.items()})

In [ ]:
import sciline as sl
from ess.nmx.loader import providers as loader_providers
from ess.nmx.loader import FileTypeMcStas, Events
from ess.nmx.reduction import providers as reduction_providers
from ess.nmx.reduction import default_params

pl = sl.Pipeline([*loader_providers, *reduction_providers], params={**params, **default_params})
file_type = FileTypeMcStas
graph = pl.get(Events[FileTypeMcStas]).visualize(graph_attr={'rankdir': 'LR'})
graph.render('example', 'png')
graph

In [ ]:
# from ess.nmx.loader import FileTypeMcStas, Events, TList, IDList
# results = pl.get((TList[file_type], IDList[file_type], Events[file_type])).compute()
# t_list = results[TList[file_type]].value
# id_list = results[IDList[file_type]].value
# da = results[Events[file_type]].value

# print("t-list range: [", t_list.min().value, ", ", t_list.max().value, "]")
# print("id-list range: [", id_list.min().value, ", ", id_list.max().value, "]")

In [ ]:
# da

In [ ]:
from ess.nmx.reduction import Grouped

graph = pl.get(Grouped[file_type]).visualize(graph_attr={'rankdir': 'LR'})
graph.render('example', format='png')
graph

In [ ]:
# grouped = pl.get(Grouped[file_type]).compute()
# grouped

In [ ]:
from ess.nmx.reduction import TimeBinned

graph = pl.get(TimeBinned[file_type]).visualize(graph_attr={'rankdir': 'LR'})
graph.render('time_binning_graph', format='png')
graph

In [ ]:
# time_binned = pl.get(TimeBinned[file_type]).compute()
# time_binned